# Worksheet 16

Name:  Esther Choi
UID: U24585295

### Topics

- Support Vector Machines (Non-linear case)

## Support Vector Machines

Follow along in class to implement the perceptron algorithm and create an animation of the algorithm.

a) As we saw in class, the form
$$w^T x + b = 0$$
while simple, does not expose the inner product `<x_i, x_j>` which we know `w` depends on, having done the math. This is critical to applying the "kernel trick" which allows for learning non-linear decision boundaries. Let's modify the above algorithm to use the form
$$\sum_i \alpha_i <x_i, x> + b = 0$$

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

epochs = 100
learning_rate = .05
expanding_rate = .99
retracting_rate = 1.1

X, labels = datasets.make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array(list(map(lambda x : -1 if x == 0 else 1, labels.tolist())))

alpha_i = np.zeros((len(X),))
b = 0

def snap(x, alpha_i, b, error):
    # create a mesh to plot in
    h = .01  # step size in the mesh
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    meshData = np.c_[xx.ravel(), yy.ravel()]
    cs = np.array([x for x in 'gb'])
    fig, ax = plt.subplots()
    ax.scatter(X[:,0],X[:,1],color=cs[labels].tolist(), s=50, alpha=0.8)

    if error:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='r',fill=False))
    else:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='y',fill=False))
   
    Z = predict_many(alpha_i, b, meshData)
    Z = np.array([0 if z <=0 else 1 for z in Z]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=.5, cmap=plt.cm.Paired)
    fig.savefig(TEMPFILE)
    plt.close()
    return im.fromarray(np.asarray(im.open(TEMPFILE)))

def predict_many(alpha_i, b, Z):
    res = []
    for i in range(len(Z)):
        res.append(predict(alpha_i, b, Z[i]))
    return np.array(res)

def predict(alpha_i, b, x):
    return np.sign(np.sum(alpha_i * Y * np.dot(X, x)) + b)

images = []
for _ in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0, len(X))
    error = False
    x, y = X[i], Y[i]
    
    if y * predict(alpha_i, b, x) <= 0:
        alpha_i[i] += learning_rate
        b += learning_rate * y
        error = True
        
    images.append(snap(x, alpha_i, b, error))

images[0].save(
    'svm_dual.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=100
)

Write a configurable kernel function to apply in lieu of the dot product. Try it out on a dataset that is not linearly separable.

In [ ]:
def polynomial(x_i, x_j, c, n):
    return (np.dot(x_i, x_j) + c) ** n


In [ ]:
import numpy as np

def gaussian_kernel(x_i, x_j, sigma=1):
    diff = np.subtract(x_i, x_j)
    squared_norm = np.dot(diff, diff)
    return np.exp(-squared_norm / (2 * sigma ** 2))

X = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
y = np.array([0, 0, 1, 1])

K = np.zeros((len(X), len(X)))
for i in range(len(X)):
    for j in range(len(X)):
        K[i, j] = gaussian_kernel(X[i], X[j], sigma=1)

# Display kernel matrix
print("Kernel Matrix:")
print(K)


b) Assume we fit an SVM using a polynomial Kernel function and it seems to overfit the data. How would you adjust the tuning parameter `n` of the kernel function?

Decrease the value of n: Reduce the degree of the polynomial kernel. Lower values of n result in simpler decision boundaries, which are less likely to overfit the training data.

c) Assume we fit an SVM using a RBF Kernel function and it seems to underfit the data. How would you adjust the tuning parameter `sigma` of the kernel function?

Increase the value of sigma: Increase the width of the Gaussian kernel. A larger sigma results in smoother decision boundaries, allowing the model to capture more complex patterns in the data. By increasing sigma, you allow the influence of each training point to extend further, leading to more flexible decision boundaries.

d) Tune the parameter of a specific Kernel function, to fit an SVM (using your code above) to the following dataset:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

data = np.loadtxt("spiral.data")
x, y = data[:, :2], data[:, 2]

plt.scatter(x[:,0], x[:,1], c=y)

In [ ]:
from sklearn.metrics import accuracy_score

K = np.zeros((len(x), len(x)))
sigma = 0.5  # Adjust this value to tune sigma
for i in range(len(x)):
    for j in range(len(x)):
        K[i, j] = gaussian_kernel(x[i], x[j], sigma=sigma)

# Define SVM with Gaussian kernel
svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='precomputed'))  # Use 'precomputed' kernel
])

svm.fit(K, y)

# Predict labels
y_pred = svm.predict(K)

# Compute accuracy
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

In [ ]:
x_min, x_max = x[:, 0].min() - 0.1, x[:, 0].max() + 0.1
y_min, y_max = x[:, 1].min() - 0.1, x[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                     np.linspace(y_min, y_max, 500))
Z = svm.predict(np.array([[gaussian_kernel(np.array([xi, yi]), np.array([xj, yj]), sigma=sigma) 
                           for xi, yi in zip(xx.ravel(), yy.ravel())] 
                          for xj, yj in x])).reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(x[:, 0], x[:, 1], c=y, edgecolors='k')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('SVM Decision Boundary (Gaussian Kernel)')
plt.show()